In [1]:
%matplotlib inline
import sys

import cv2
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.svm import SVC
from sklearn import cross_validation

path = 'LP_data/dataset/P{}/G{}/R{}_l.png'

/usr/local/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
training_paths = []
labels = []
for p in range(1, 8):
    for g in range(1, 11):
        for r in range(1, 21):
            fname = path.format(p, g, r)
            training_paths.append(fname)
            labels.append(g)
print(len(labels))

1400


In [3]:
sift = cv2.xfeatures2d.SIFT_create()
BOW = cv2.BOWKMeansTrainer(10)
total = len(training_paths)
for current, p in enumerate(training_paths):
    sys.stdout.flush()
    sys.stdout.write('Processing {} / {}\r'.format(current+1, total))
    img = cv2.imread(p)
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    kp, dsc = sift.detectAndCompute(gray, None)
    BOW.add(dsc)
    
dictionary = BOW.cluster()

In [4]:
FLANN_INDEX_KDTREE = 0
index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
search_params = dict(checks=50)
flann = cv2.FlannBasedMatcher(index_params, search_params)
sift2 = cv2.xfeatures2d.SIFT_create()
bowDiction = cv2.BOWImgDescriptorExtractor(sift2, cv2.BFMatcher(cv2.NORM_L2))
bowDiction.setVocabulary(dictionary)
print("bow dictionary: {}".format(np.shape(dictionary)))

bow dictionary: (10, 128)


In [5]:
def feature_extract(p):
    img = cv2.imread(p, 1)
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    return bowDiction.compute(gray, sift.detect(gray))

In [6]:
train_desc = []
for current, p in enumerate(training_paths):
    sys.stdout.flush()
    sys.stdout.write('Processing {} / {}\r'.format(current+1, total))
    train_desc.extend(feature_extract(p))
    
print('\nSVM items: {} {}'.format(len(train_desc), len(train_desc[0])))

Processing 1400 / 1400
SVM items: 1400 10


In [7]:
svm = SVC()

svm.fit(np.array(train_desc), np.array(labels))

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [8]:
clf = SVC()
n_cv = 5
res = cross_validation.cross_val_score(clf, train_desc, labels, cv=n_cv, scoring='accuracy')
print()
print('Mean: {:>5.2f}'.format(res.mean()))
print('Max: {:>5.2f}'.format(res.max()))
print('Min: {:>5.2f}'.format(res.min()))


Mean:  0.33
Max:  0.35
Min:  0.30


In [9]:
confusion = np.zeros((10, 10))
def classify(p, cnt):
    feature = feature_extract(p)
    p = svm.predict(feature)
    confusion[labels[cnt]-1, p-1] += 1

for current, p in enumerate(training_paths):
    sys.stdout.flush()
    sys.stdout.write('Processing {} / {}\r'.format(current+1, total))
    classify(p, current)

KeyboardInterrupt: 

In [ ]:
def normalize_rows(mat):
    rows_sum = mat.sum(axis=1)
    return mat / rows_sum

In [ ]:
confusion_normalized = normalize_rows(confusion).transpose()

sns.heatmap(confusion_normalized, annot=True, fmt='.2f')